# Prática Supervisionada

In [1]:
import pandas as pd
from sklearn import model_selection
from sklearn import preprocessing

## Carregue o dataset wine.data

In [5]:
df = pd.read_csv('wine.data', header=None)
df[0].unique()




array([1, 2, 3])

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


# Dividir os dados entre treino e teste de forma estratificada

In [6]:
df.isnull().sum()




0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
0     178 non-null int64
1     178 non-null float64
2     178 non-null float64
3     178 non-null float64
4     178 non-null float64
5     178 non-null int64
6     178 non-null float64
7     178 non-null float64
8     178 non-null float64
9     178 non-null float64
10    178 non-null float64
11    178 non-null float64
12    178 non-null float64
13    178 non-null int64
dtypes: float64(11), int64(3)
memory usage: 19.5 KB


In [32]:
X = df.values[:, 1:]
y = df.values[:, 0]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                            stratify=y, test_size=0.30, random_state=42) 

In [33]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((124, 13), (54, 13), (124,), (54,))

# Provar a estratificação dos dados com relação aos labels

In [34]:
import numpy as np
count_y_train = pd.Series(y_train).value_counts()
count_y_test = pd.Series(y_test).value_counts()


print(df[0].value_counts()/np.sum(df[0].value_counts()))
print(count_y_train/np.sum(count_y_train))
print(count_y_test/np.sum(count_y_test))


2    0.398876
1    0.331461
3    0.269663
Name: 0, dtype: float64
2.0    0.403226
1.0    0.330645
3.0    0.266129
dtype: float64
2.0    0.388889
1.0    0.333333
3.0    0.277778
dtype: float64


# Usar Grid Search com validação cruzada 5-Fold e diferentes parâmetros para diversos modelos e encontrar o melhor deles. Aplicar Standardization nos dados.

In [35]:
from sklearn import svm

In [39]:
from sklearn import pipeline
ppl = pipeline.make_pipeline(preprocessing.StandardScaler(), 
                                  svm.SVC())

In [37]:
params = [{
    'svc__C': [10000, 0.001],
    'svc__kernel': ['linear', 'rbf']
}]

In [45]:
search_clf = model_selection.GridSearchCV(
    ppl, params, n_jobs=1, cv=5, verbose=2, iid=False
)

In [46]:
search_clf.fit(X_train, y_train
            )

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] svc__C=10000, svc__kernel=linear ................................
[CV] ................. svc__C=10000, svc__kernel=linear, total=   0.0s
[CV] svc__C=10000, svc__kernel=linear ................................
[CV] ................. svc__C=10000, svc__kernel=linear, total=   0.0s
[CV] svc__C=10000, svc__kernel=linear ................................
[CV] ................. svc__C=10000, svc__kernel=linear, total=   0.0s
[CV] svc__C=10000, svc__kernel=linear ................................
[CV] ................. svc__C=10000, svc__kernel=linear, total=   0.0s
[CV] svc__C=10000, svc__kernel=linear ................................
[CV] ................. svc__C=10000, svc__kernel=linear, total=   0.0s
[CV] svc__C=10000, svc__kernel=rbf ...................................
[CV] .................... svc__C=10000, svc__kernel=rbf, total=   0.0s
[CV] svc__C=10000, svc__kernel=rbf ...................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('svc',
                                        SVC(C=1.0, cache_size=200,
                                            class_weight=None, coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='auto_deprecated',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False))],
        

In [47]:
search_clf.best_params_

{'svc__C': 10000, 'svc__kernel': 'rbf'}

In [48]:
search_clf.best_score_

0.992

In [49]:
search_clf.score(X_test, y_test)

0.9814814814814815

In [73]:
y_pred = search_clf.best_estimator_.predict(X_test)

In [80]:
from sklearn import metrics
metrics.confusion_matrix(y_test, y_pred, labels=[1, 2, 3])

array([[18,  0,  0],
       [ 0, 21,  0],
       [ 0,  1, 14]])

In [81]:
print(metrics.classification_report(y_test, y_pred, labels=[1,2,3]))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        18
           2       0.95      1.00      0.98        21
           3       1.00      0.93      0.97        15

    accuracy                           0.98        54
   macro avg       0.98      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54



In [60]:
model = pipeline.make_pipeline(preprocessing.StandardScaler(),
                          svm.SVC(C=10000, kernel='rbf'))

In [61]:
print(model)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)


In [62]:
model.fit(X, y)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [57]:
X[0
]

array([1.423e+01, 1.710e+00, 2.430e+00, 1.560e+01, 1.270e+02, 2.800e+00,
       3.060e+00, 2.800e-01, 2.290e+00, 5.640e+00, 1.040e+00, 3.920e+00,
       1.065e+03])

In [69]:
linha = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13])

In [70]:
X[0].shape

(13,)

In [71]:
model.predict(linha.reshape(1,13))

array([2.])